# PART 1: READ AND CLEAN DATA

# Date

In [ ]:
date = '17-May'

In [ ]:
import pandas as pd
pd.set_option('chained_assignment', None)

df = pd.read_csv("S:\\Office of Risk Management\\User Folders\Darius Mehri\\
Enforcement Inspection Analysis\\Bobby Allesio Analysis\\April 2017\\
Time Entry Detail Report Mar 17.csv")

df["First Name"] = df["First Name"].map(str.upper)
df["Last Name"] = df["Last Name"].map(str.upper)
df["First Name"] = df["First Name"].map(str.strip)
df["Last Name"] = df["Last Name"].map(str.strip)


df["Last Name"] = df["Last Name"].str.replace(' JR', '')
df["Last Name"] = df["Last Name"].str.replace(',', '')
df["Last Name"] = df["Last Name"].str.replace('.', '')
df["Last Name"] = df["Last Name"].str.replace(' II', '')
df["Last Name"] = df["Last Name"].str.replace(' III', '')

df["Name"] = df["First Name"] + " " + df["Last Name"]

df["Actual In Punch"] = df["Actual In Punch"].astype(str)
df["Actual In Punch"] = df["Actual In Punch"].map(str.strip)

df["Actual Out Punch"] = df["Actual Out Punch"].astype(str)
df["Actual Out Punch"] = df["Actual Out Punch"].map(str.strip)

#BOBBY ALESIO CALCULATION, GET PEOPLE OF INTEREST FIRST
bn = pd.read_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Enforcement Inspection Analysis\\Bobby Allesio Analysis\\Inspector List 3.csv")
bn["FIRST NAME"] = bn["FIRST NAME"].map(str.upper)
bn["LAST NAME"] = bn["LAST NAME"].map(str.upper)
bn["FIRST NAME"] = bn["FIRST NAME"].map(str.strip)
bn["LAST NAME"] = bn["LAST NAME"].map(str.strip)

bn["LAST NAME"] = bn["LAST NAME"].str.replace(' JR', '')
bn["LAST NAME"] = bn["LAST NAME"].str.replace(',', '')
bn["LAST NAME"] = bn["LAST NAME"].str.replace('.', '')
bn["LAST NAME"] = bn["LAST NAME"].str.replace(' II', '')
bn["LAST NAME"] = bn["LAST NAME"].str.replace(' III', '')

bn["Name"] = bn["FIRST NAME"] + " " + bn["LAST NAME"]

#create list of names
nameList = bn['Name'].tolist()
nameList = set(nameList)
nameList = list(nameList)

print len(nameList)

#subset df using list of names
df = df[df['Name'].isin(nameList)]
df = df.reset_index(drop=True)
namedfList = df['Name'].tolist()
namedfList = set(namedfList)
namedfList = list(namedfList)

print len(namedfList)

#get list of names with blank for punch in and/or punch out

df = df.reset_index(drop=True)
print "PUNCH IN TIME BLANK"
for i in range(0, len(df)):
    if df["Actual In Punch"][i] == 'nan':
        print df["Last Name"][i], df["First Name"][i], df["Employee CT ID"][i], df["Date of In Punch"][i]

print
print "PUNCH OUT TIME BLANK"

for i in range(0, len(df)):
    if df["Actual Out Punch"][i] == 'nan':
        print df["Last Name"][i], df["First Name"][i], df["Employee CT ID"][i], df["Date of In Punch"][i]



In [ ]:
len(bn)

In [ ]:
#check who not in list
for i in range(0, len(nameList)):
    if nameList[i] not in namedfList:
        print nameList[i]

In [ ]:
#correct for time punch where have two rows of data, theoretically should have only one row

print len(df)

df['Blank Punch'] = '0'

#df["Actual In Punch"] = df["Actual In Punch"].astype(str)
df["Actual In Punch"] = df["Actual In Punch"].map(str.strip)

#df["Actual Out Punch"] = df["Actual Out Punch"].astype(str)
df["Actual Out Punch"] = df["Actual Out Punch"].map(str.strip)


#this corrects for have punch in but no punch out, and punch out but no punch in
for i in range(0, len(df)):
    if (df["Actual In Punch"][i] == 'nan'):
        df["Actual In Punch"][i] = "9:00"
        df["Actual Out Punch"][i] = "17:00"
        
    if df["Actual Out Punch"][i] == 'nan':
        df["Actual In Punch"][i] = "9:00"
        df["Actual Out Punch"][i] = "17:00"
        df['Blank Punch'][i] = '1'

print df["Actual In Punch"].isnull().sum()
print df["Actual Out Punch"].isnull().sum()


print "len before drop", len(df)
df = df[df["Blank Punch"] != '1']
df = df.reset_index(drop=True)
print "len after drop", len(df)


#convert to datetime
df['Actual In Punch'] = pd.to_datetime(df['Actual In Punch'])
df['Actual Out Punch'] = pd.to_datetime(df['Actual Out Punch'])
#strip date from datetime
df['Actual In Punch'] = df["Actual In Punch"].apply( lambda d : d.time() )
df['Actual Out Punch'] = df["Actual Out Punch"].apply( lambda d : d.time() )

#drop duplicates
#print "len before drop", len(df)
#df = df.drop_duplicates(subset=['Employee CT ID', 'Date of In Punch', 'Name']).reset_index(drop=True)
#df = df.reset_index(drop=True)

#print "len after drop", len(df)



# PART 2: CALCULATE HOURS WORKED

In [ ]:
pd.set_option('chained_assignment', None)

df["Hours Worked"] = ""

for i in range(0, len(df)):
    d = (df["Actual Out Punch"][i].hour + df["Actual Out Punch"][i].minute/60.0) \
        - (df["Actual In Punch"][i].hour + df["Actual In Punch"][i].minute/60.0)
    df["Hours Worked"][i] = d

# PART 3: REMOVE HOURS FOR LUNCH

In [ ]:
#categorize as inspector
#bn["Hours"] = bn["Hours"].astype(str)

df2 = df[["Name", "Date of In Punch", "Hours Worked"]]

dfG = df2.groupby(['Name', 'Date of In Punch']).sum()

dfG = dfG.add_suffix('_Count').reset_index()

dfG = dfG.rename(columns={'Hours Worked_Count': 'Hours Worked'})
dfG = dfG.reset_index(drop=True)

#create inspector list, inspectors only take 1/2 hour for lunch
insp = bn[bn["Hours"] == 8]

inspList = insp['Name'].tolist()

print len(inspList)
print len(bn)

print inspList

dfG["Inspector"] = 0

for i in range(0, len(dfG)):
    if dfG["Name"][i] in inspList:
        dfG["Inspector"][i] = 1

#correct for lunch

for i in range(0, len(dfG)):
    if dfG["Inspector"][i] == 1:
        dfG["Hours Worked"][i] = dfG["Hours Worked"][i] - 0.5
    else:
        dfG["Hours Worked"][i] = dfG["Hours Worked"][i] - 1.0

        
#add up hours worked for each employee
dfG2 = (dfG.groupby('Name').agg({'Hours Worked':'sum'}).reset_index().rename(columns={'Hours Worked':'Actual Hours Worked'}) )
dfG2 = dfG2.rename(columns={'Actual Hours Worked': date})


In [ ]:
dfG2.to_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Enforcement Inspection Analysis\\
Bobby Allesio Analysis\\April 2017\\Actual Hours Worked March 2017.csv")

In [ ]:
dfG2.head(2)

In [ ]:
#merged = merged[["Name", "LAST NAME", "FIRST NAME", "TITLE", "UNIT", "Hours"]]
merged = bn
merged = pd.merge(merged,dfG2, how="outer" )

merged

In [ ]:
merged.to_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Enforcement Inspection Analysis\\
Bobby Allesio Analysis\\April 2017\\Alesio Actual Hours Worked.csv")